## Analyse Z-coordinates between surfactant and slab

### Please identify input directory

In [74]:
molecule = 'UNL'
gro_file_path = '/home/newsitt/Desktop/dlic_20ns/prep_conf.gro'
output_gro_file = "/home/newsitt/Desktop/dlic_20ns/conf.gro"
z_offset = -0.6  # Amount to shift Z-coordinates

# gro_file_path = "/home/newsitt/Desktop/dlic_20ns/prep_conf.gro"
original_txt_file = "surf_initial_coor.txt"
edited_txt_file = "surf_final_coor.txt"


### Return distance between surfactant molecule and surface 'BEFORE' translating

In [75]:
import numpy as np

# 📌 Step 1: Load the GRO file
def load_gro_file(file_path):
    """Reads the contents of a GROMACS .gro file."""
    with open(file_path, 'r') as file:
        content = file.readlines()
    return content

# 📌 Step 2: Extract lines corresponding to a specific residue
def extract_residue_lines(content, residue_name):
    """Extracts atom lines belonging to the specified residue in a .gro file."""
    residue_lines = []
    for line in content[2:-1]:  # Skip title and number of atoms
        if len(line) < 20:  # Skip invalid lines
            continue
        resname = line[5:10].strip()  # Extract the residue name from the fixed-width format
        if resname == residue_name:
            residue_lines.append(line)
    return residue_lines

# 📌 Step 3: Parse atom information and calculate average, max, and min coordinates
def analyze_coordinates(residue_lines):
    """Calculates the average, max, and min Z-coordinates for a given residue."""
    if not residue_lines:
        print("⚠️ Warning: No atoms found for this residue!")
        return None

    # Extract atom coordinates
    coordinates = np.array([
        list(map(float, [line[20:28], line[28:36], line[36:44]]))  # Extract x, y, z columns
        for line in residue_lines
    ])

    # Compute average coordinates
    avg_coords = np.mean(coordinates, axis=0)
    
    # Extract Z values
    min_z = np.min(coordinates[:, 2])
    max_z = np.max(coordinates[:, 2])

    return {
        "average": {"x": avg_coords[0], "y": avg_coords[1], "z": avg_coords[2]},
        "max_z": max_z,
        "min_z": min_z
    }

# 📌 Step 4: Compare two residues and calculate distances
def compare_residues_and_calculate_distance(gro_file_path, molecule):
    """Compares the Z-coordinates of a molecule and Fe and calculates their distance."""
    gro_content = load_gro_file(gro_file_path)

    results = {}
    residues = [molecule, "Fe"]

    for residue in residues:
        residue_lines = extract_residue_lines(gro_content, residue)
        results[residue] = analyze_coordinates(residue_lines)

    # Ensure both residues were found
    if None in results.values():
        print("⚠️ Error: One or both residues were not found in the .gro file!")
        return results, None, gro_file_path

    # Compute distance between min Z of the molecule and max Z of Fe
    min_molecule_z = results[molecule]["min_z"]
    max_fe_z = results["Fe"]["max_z"]
    distance = abs(min_molecule_z - max_fe_z)

    return results, distance, gro_file_path

# 📌 Step 5: Run the analysis
if __name__ == "__main__":
    # Define molecule of interest
    #molecule = "UNL"  # Change this as needed

    # Define the GRO file path
    #gro_file_path = output_gro_file  # Replace with your .gro file

    # Perform analysis
    comparison_results, z_distance, working_file_path = compare_residues_and_calculate_distance(gro_file_path, molecule)

    # 📌 Step 6: Print the results
    print(f"\n📂 Working with file: {working_file_path}")
    
    if comparison_results and z_distance is not None:
        for residue, result in comparison_results.items():
            print(f"\n🔍 Analysis for Residue: {residue}")
            print(f"📊 Average Coordinates:")
            print(f"   x: {result['average']['x']:.3f} nm")
            print(f"   y: {result['average']['y']:.3f} nm")
            print(f"   z: {result['average']['z']:.3f} nm")
            print(f"📏 Z Direction Analysis:")
            print(f"   🔼 Max Z: {result['max_z']:.3f} nm")
            print(f"   🔽 Min Z: {result['min_z']:.3f} nm")

        print(f"\n📏 Distance between min Z of {molecule} and max Z of Fe: **{z_distance:.3f} nm**")



📂 Working with file: /home/newsitt/Desktop/dlic_20ns/prep_conf.gro

🔍 Analysis for Residue: UNL
📊 Average Coordinates:
   x: 2.943 nm
   y: 2.467 nm
   z: 1.908 nm
📏 Z Direction Analysis:
   🔼 Max Z: 2.245 nm
   🔽 Min Z: 1.440 nm

🔍 Analysis for Residue: Fe
📊 Average Coordinates:
   x: 1.477 nm
   y: 1.485 nm
   z: 0.344 nm
📏 Z Direction Analysis:
   🔼 Max Z: 0.628 nm
   🔽 Min Z: 0.060 nm

📏 Distance between min Z of UNL and max Z of Fe: **0.812 nm**


### Translating process

In [76]:
import numpy as np

# 📌 Step 1: Load the GRO file
def load_gro_file(file_path):
    """Reads the contents of a GROMACS .gro file."""
    with open(file_path, 'r') as file:
        content = file.readlines()
    return content

# 📌 Step 2: Extract lines corresponding to UNL residues
def extract_unl_residue_lines(content):
    """Extracts lines containing 'UNL' residues from a .gro file."""
    unl_lines = []
    for line in content[2:-1]:  # Skip first and last lines (title & box info)
        if len(line) < 44:  # Ensure valid .gro format
            continue
        resname = line[5:10].strip()  # Extract residue name (fixed width)
        if resname == molecule: # *** DON'T FORGET TO CHANGE THIS INPUT ***
            unl_lines.append(line)
    return unl_lines

# 📌 Step 3: Edit Z coordinates
def edit_z_coordinates(unl_lines, z_offset):
    """Modifies the Z-coordinate of UNL residues by a given offset."""
    edited_lines = []
    for line in unl_lines:
        residue_info = line[:20]  # Preserve the first 20 characters (residue ID and atom info)
        x = float(line[20:28])
        y = float(line[28:36])
        z = float(line[36:44])
        z_new = z + z_offset  # Modify Z coordinate
        edited_line = f"{residue_info}{x:8.3f}{y:8.3f}{z_new:8.3f}\n"
        edited_lines.append(edited_line)
    return edited_lines

# 📌 Step 4: Replace edited lines in the original GRO file
def replace_coordinates_in_gro(content, unl_lines, edited_unl_lines):
    """Replaces original UNL lines in the .gro file with edited ones."""
    updated_content = content.copy()
    for original, edited in zip(unl_lines, edited_unl_lines):
        index = updated_content.index(original)  # Find index of original line
        updated_content[index] = edited  # Replace with modified line
    return updated_content

# 📌 Step 5: Save the updated GRO file
def save_updated_gro_file(content, output_file):
    """Saves the updated GRO file."""
    with open(output_file, 'w') as file:
        file.writelines(content)

# 📌 Step 6: Save extracted coordinates to TXT files
def save_coordinates_to_txt(lines, output_file):
    """Saves extracted UNL coordinates to a text file."""
    with open(output_file, 'w') as txtfile:
        txtfile.write("ResidueID  AtomName     x         y         z\n")
        txtfile.write("---------- ---------- ---------- ---------- ----------\n")
        txtfile.writelines(lines)

# 📌 Main Execution
if __name__ == "__main__":
    # 📂 File paths (replace as needed)
    #gro_file_path = "/home/newsitt/Desktop/dlic_20ns/prep_conf.gro"
    #original_txt_file = "/home/newsitt/Desktop/dlic_20ns/surf_initial_coor.txt"
    #edited_txt_file = "/home/newsitt/Desktop/dlic_20ns/surf_final_coor.txt"
    #output_gro_file = "/home/newsitt/Desktop/dlic_20ns/conf.gro"
    #z_offset = -0.6  # Amount to shift Z-coordinates

    # 🏗️ Load the GRO file
    gro_content = load_gro_file(gro_file_path)

    # 🔍 Extract UNL residue lines
    unl_residue_lines = extract_unl_residue_lines(gro_content)
    
    if not unl_residue_lines:
        print("⚠️ Warning: No UNL residues found in the GRO file!")
        exit()

    # 📄 Save original coordinates to TXT
    save_coordinates_to_txt(unl_residue_lines, original_txt_file)

    # ✏️ Edit Z coordinates
    edited_unl_lines = edit_z_coordinates(unl_residue_lines, z_offset)

    # 📄 Save edited coordinates to TXT
    save_coordinates_to_txt(edited_unl_lines, edited_txt_file)

    # 🔄 Replace edited coordinates in GRO content
    updated_gro_content = replace_coordinates_in_gro(gro_content, unl_residue_lines, edited_unl_lines)

    # 💾 Save the updated GRO file
    save_updated_gro_file(updated_gro_content, output_gro_file)

    # 📌 Recap Outputs
    recap_message = f"""
    🔄 Recap of Actions:
    1️⃣ Extracted original coordinates of UNL residues from **{gro_file_path}**.
       - 📄 Saved to: **{original_txt_file}**

    2️⃣ Edited Z-coordinates of UNL residues by adding **{z_offset}**.
       - 📄 Saved edited coordinates to: **{edited_txt_file}**

    3️⃣ Replaced the edited coordinates back into the GRO file.
       - 📄 Updated GRO file: **{output_gro_file}**

    ✅ Next Steps:
    - 🏗️ **Review extracted & edited coordinates** in the TXT files.
    - 🖥️ **Visualize updated GRO file** (`{output_gro_file}`) in a molecular viewer.
    """
    print(recap_message)



    🔄 Recap of Actions:
    1️⃣ Extracted original coordinates of UNL residues from **/home/newsitt/Desktop/dlic_20ns/prep_conf.gro**.
       - 📄 Saved to: **surf_initial_coor.txt**

    2️⃣ Edited Z-coordinates of UNL residues by adding **-0.6**.
       - 📄 Saved edited coordinates to: **surf_final_coor.txt**

    3️⃣ Replaced the edited coordinates back into the GRO file.
       - 📄 Updated GRO file: **/home/newsitt/Desktop/dlic_20ns/conf.gro**

    ✅ Next Steps:
    - 🏗️ **Review extracted & edited coordinates** in the TXT files.
    - 🖥️ **Visualize updated GRO file** (`/home/newsitt/Desktop/dlic_20ns/conf.gro`) in a molecular viewer.
    


### Return distance between surfactant molecule and surface 'AFTER' translating

In [77]:
import numpy as np

# 📌 Step 1: Load the GRO file
def load_gro_file(file_path):
    """Reads the contents of a GROMACS .gro file."""
    with open(file_path, 'r') as file:
        content = file.readlines()
    return content

# 📌 Step 2: Extract lines corresponding to a specific residue
def extract_residue_lines(content, residue_name):
    """Extracts atom lines belonging to the specified residue in a .gro file."""
    residue_lines = []
    for line in content[2:-1]:  # Skip title and number of atoms
        if len(line) < 20:  # Skip invalid lines
            continue
        resname = line[5:10].strip()  # Extract the residue name from the fixed-width format
        if resname == residue_name:
            residue_lines.append(line)
    return residue_lines

# 📌 Step 3: Parse atom information and calculate average, max, and min coordinates
def analyze_coordinates(residue_lines):
    """Calculates the average, max, and min Z-coordinates for a given residue."""
    if not residue_lines:
        print("⚠️ Warning: No atoms found for this residue!")
        return None

    # Extract atom coordinates
    coordinates = np.array([
        list(map(float, [line[20:28], line[28:36], line[36:44]]))  # Extract x, y, z columns
        for line in residue_lines
    ])

    # Compute average coordinates
    avg_coords = np.mean(coordinates, axis=0)
    
    # Extract Z values
    min_z = np.min(coordinates[:, 2])
    max_z = np.max(coordinates[:, 2])

    return {
        "average": {"x": avg_coords[0], "y": avg_coords[1], "z": avg_coords[2]},
        "max_z": max_z,
        "min_z": min_z
    }

# 📌 Step 4: Compare two residues and calculate distances
def compare_residues_and_calculate_distance(gro_file_path, molecule):
    """Compares the Z-coordinates of a molecule and Fe and calculates their distance."""
    gro_content = load_gro_file(gro_file_path)

    results = {}
    residues = [molecule, "Fe"]

    for residue in residues:
        residue_lines = extract_residue_lines(gro_content, residue)
        results[residue] = analyze_coordinates(residue_lines)

    # Ensure both residues were found
    if None in results.values():
        print("⚠️ Error: One or both residues were not found in the .gro file!")
        return results, None, gro_file_path

    # Compute distance between min Z of the molecule and max Z of Fe
    min_molecule_z = results[molecule]["min_z"]
    max_fe_z = results["Fe"]["max_z"]
    distance = abs(min_molecule_z - max_fe_z)

    return results, distance, gro_file_path

# 📌 Step 5: Run the analysis
if __name__ == "__main__":
    # Define molecule of interest
    molecule = "UNL"  # Change this as needed

    # Define the GRO file path
    gro_file_path = output_gro_file  # Replace with your .gro file

    # Perform analysis
    comparison_results, z_distance, working_file_path = compare_residues_and_calculate_distance(gro_file_path, molecule)

    # 📌 Step 6: Print the results
    print(f"\n📂 Working with file: {working_file_path}")
    
    if comparison_results and z_distance is not None:
        for residue, result in comparison_results.items():
            print(f"\n🔍 Analysis for Residue: {residue}")
            print(f"📊 Average Coordinates:")
            print(f"   x: {result['average']['x']:.3f} nm")
            print(f"   y: {result['average']['y']:.3f} nm")
            print(f"   z: {result['average']['z']:.3f} nm")
            print(f"📏 Z Direction Analysis:")
            print(f"   🔼 Max Z: {result['max_z']:.3f} nm")
            print(f"   🔽 Min Z: {result['min_z']:.3f} nm")

        print(f"\n📏 Distance between min Z of {molecule} and max Z of Fe: **{z_distance:.3f} nm**")



📂 Working with file: /home/newsitt/Desktop/dlic_20ns/conf.gro

🔍 Analysis for Residue: UNL
📊 Average Coordinates:
   x: 2.943 nm
   y: 2.467 nm
   z: 1.308 nm
📏 Z Direction Analysis:
   🔼 Max Z: 1.645 nm
   🔽 Min Z: 0.840 nm

🔍 Analysis for Residue: Fe
📊 Average Coordinates:
   x: 1.477 nm
   y: 1.485 nm
   z: 0.344 nm
📏 Z Direction Analysis:
   🔼 Max Z: 0.628 nm
   🔽 Min Z: 0.060 nm

📏 Distance between min Z of UNL and max Z of Fe: **0.212 nm**


### Visualizing results

In [78]:
import py3Dmol

def read_gro_file_with_box_and_units(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Converts units from nanometers to angstroms for py3Dmol visualization.
    Returns:
        - gro_data: The entire contents of the .gro file as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    box_dims = (0.0, 0.0, 0.0)
    try:
        with open(filename, "r") as file:
            gro_data = file.read()
            
            # Extract the last line for box dimensions
            lines = gro_data.strip().split("\n")
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                # Convert box dimensions from nm to Å
                box_dims = tuple(float(dim) * 10 for dim in box_line[:3])
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                print("Box dimensions line is invalid.")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    return gro_data, box_dims

def visualize_gro_with_box_units(gro_data, box_dims):
    """
    Visualizes the .gro file with simulation box edges using py3Dmol.
    Converts the box dimensions and coordinates to angstroms.
    Args:
        - gro_data: The .gro file contents as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    if box_dims == (0.0, 0.0, 0.0):
        print("Invalid box dimensions. Cannot draw simulation box.")
        return

    viewer = py3Dmol.view(width=800, height=600)
    
    # Add the molecular structure from the .gro file
    viewer.addModel(gro_data, "gro")  # Load .gro file
    viewer.setStyle({"sphere": {"scale": 0.3}})  # Atoms as spheres
    
    # Add the box edges (in Å)
    x_len, y_len, z_len = box_dims
    box_lines = [
        [[0, 0, 0], [x_len, 0, 0]], [[x_len, 0, 0], [x_len, y_len, 0]],
        [[x_len, y_len, 0], [0, y_len, 0]], [[0, y_len, 0], [0, 0, 0]],  # Bottom face
        [[0, 0, z_len], [x_len, 0, z_len]], [[x_len, 0, z_len], [x_len, y_len, z_len]],
        [[x_len, y_len, z_len], [0, y_len, z_len]], [[0, y_len, z_len], [0, 0, z_len]],  # Top face
        [[0, 0, 0], [0, 0, z_len]], [[x_len, 0, 0], [x_len, 0, z_len]],
        [[x_len, y_len, 0], [x_len, y_len, z_len]], [[0, y_len, 0], [0, y_len, z_len]]  # Vertical edges
    ]
    for line in box_lines:
        viewer.addCylinder({"start": {"x": line[0][0], "y": line[0][1], "z": line[0][2]},
                            "end": {"x": line[1][0], "y": line[1][1], "z": line[1][2]},
                            "radius": 0.1, "color": "gray"})

    # Center and zoom the viewer
    viewer.zoomTo()
    
    # Show the viewer
    return viewer.show()

def main():
    # Input .gro file
    # gro_filename = "/home/newsitt/Desktop/dlic_20ns/prep_conf.gro"
    gro_filename = output_gro_file
    
    # Read the .gro file and box dimensions
    gro_data, box_dims = read_gro_file_with_box_and_units(gro_filename)
    
    # Visualize the .gro file with the simulation box
    visualize_gro_with_box_units(gro_data, box_dims)

if __name__ == "__main__":
    main()

Extracted box dimensions (in Å): (30.0, 30.0, 50.0)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.